# 🌀 OSM integration

In [ ]:
from blueflow import notebooks

from pyproj import Transformer

from blue_options import string
from blue_objects import objects, file
from blue_objects.metadata import get_from_object

from palisades.geo.OSM import *
from palisades.host import signature
from palisades.logger import logger

logger.info(f"{'.'.join(signature())},\nbuilt on {string.pretty_date()}")

In [2]:
object_name = "predict-datacube-maxar_open_data-WildFires-LosAngeles-Jan-2025-11-031311102212-10400100A0B73800-2025-01-21-jeko6i"

---

In [3]:
assert objects.download(object_name)
metadata = get_from_object(object_name, "predict")

In [4]:
geoimage_path = objects.path_of(
    filename=metadata["output_filename"],
    object_name=object_name,
)
output_geojson_path = file.add_extension(geoimage_path,"geojson")

In [ ]:
# 1. Load Fire Damage Assessment Layer
damage_layer, affine, crs, bbox = load_fire_damage_layer(geoimage_path)

logger.info(f"crs: {crs}, bbox:{bbox}")

In [ ]:
transformer = Transformer.from_crs(crs, "EPSG:4326", always_xy=True)

min_x, min_y = transformer.transform(bbox[0], bbox[1])
max_x, max_y = transformer.transform(bbox[2], bbox[3])

bbox_wgs84 = (min_y, max_y, min_x, max_x)  # Format as (south, north, west, east)
logger.info(f"bbox_wgs84: {bbox_wgs84}")


In [ ]:
buildings = ox.features_from_bbox(
    bbox=bbox_wgs84,
    tags={"building": True},
)


In [ ]:
gdf =  buildings[buildings.geometry.notnull()].reset_index()

---

In [ ]:
# assert notebooks.upload(object_name)

In [5]:
# END